In [320]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

In [321]:
path = 'reconsile_payment.xlsx'
remit_file = 'Remittances.xlsx'
lookup_file = 'CDMS Lookups.xlsx'
df = pd.read_excel(path, engine='openpyxl')
df_remit = pd.read_excel(remit_file, engine='openpyxl')
df_lookup = pd.read_excel(lookup_file, engine='openpyxl', sheet_name='PO Approvers')

C:\Users\divyal\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:214: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")


In [322]:
df_lookup_new = df_lookup.iloc[:,:2]


In [323]:
df_billable = df.loc[df['Skil_Eqip_Mat'] != 'Nonbillable'].copy()


In [324]:
def module (row):
    if row["Module"] == "Material":
        return row["OT_Hours"]
    else:
        return row["ST_Rate"] * row["ST_Hours"] + row["OT_Rate"] * row["OT_Hours"] + row["DT_Hours"] * row["DT_Rate"]


def auth (row):
    if row["timesheet_reference"] > 0:
        return "Authorized"
    else:
        return "Unauthorized"


def status (row):
    if row["Paid"] == "OutStanding":
        return "OutStanding"
    else:
        return "Paid"


def week_ending(row):
    ini_date = pd.to_datetime(row["TS_Int"])
    new_date = timedelta((12- ini_date.weekday()) % 7)
    week_end_date = ini_date + new_date
    return week_end_date

def emp_unit(row):
    if row["Module"] == "Labour":
        return f'{row["Name1"]} {row["Name2"]}'

    elif row["Module"] == "Equipment":
        return f'{row["employee_or_equipment_id"]} {row["Name1"]}'

    elif row["Module"] == "Material":
        return row["Name1"]
    
    else:
        return "Check"


In [325]:
df_billable['Line Cost'] = df_billable.apply(lambda row: module (row), axis=1)


In [326]:
df_billable['Authorized'] = df_billable.apply(lambda row: auth (row), axis=1)

In [327]:
df_billable['Week Ending'] = df_billable.apply(lambda row: week_ending (row), axis=1)

In [328]:
new_header = df_remit.iloc[2] #grab the first row for the header
df_remit_new = df_remit[3:] #take the data less the header row
df_remit_new.columns = new_header #set the header row as the df header


In [329]:
df_temp = pd.DataFrame(df_remit_new[['Amount', 'Date Paid']]).copy()

df_temp.rename(columns={'Amount': 'timesheet_reference'}, inplace=True)
df_temp
df_temp.dropna(inplace=True)


In [330]:
df_billable['timesheet_reference'].round(2)

0             NaN
1             NaN
2             NaN
3             0.0
4             0.0
          ...    
2548    4788110.0
2549    4788110.0
2550    4788110.0
2551    4788110.0
2552    4791847.0
Name: timesheet_reference, Length: 1696, dtype: float64

In [331]:
df_billable = pd.merge(df_billable, 
                     df_temp, 
                     on ='timesheet_reference', 
                     how ='left')

In [332]:
df_billable['Date Paid'].replace(np.nan, 'OutStanding', inplace=True)

In [333]:
df_billable.rename(columns={'Date Paid': 'Paid'}, inplace=True)

In [334]:
df_billable['Status'] = df_billable.apply(lambda row: status (row), axis=1)


In [335]:
df_billable['Employee/Unit'] = df_billable.apply(lambda row: emp_unit (row), axis=1)

In [336]:
df_lookup_new.rename(columns={'Area ID': 'area_id', 'Approver Name': 'Approver'}, inplace=True)

In [337]:
df_billable = pd.merge(df_billable, 
                     df_lookup_new, 
                     on ='area_id', 
                     how ='left')


In [338]:
auth_column = df_billable.pop('Authorized')
df_billable.insert(0, 'Authorized', auth_column)

In [339]:
status_column = df_billable.pop('Status')
df_billable.insert(0, 'Status', status_column)

In [340]:
paid_column = df_billable.pop('Paid')
df_billable.insert(0, 'Paid', paid_column)

In [341]:
week_ending_column = df_billable.pop('Week Ending')
df_billable.insert(0, 'Week Ending', week_ending_column)

In [342]:
emp_unit_column = df_billable.pop('Employee/Unit')
df_billable.insert(0, 'Employee/Unit', emp_unit_column)

In [343]:
approver_column = df_billable.pop('Approver')
df_billable.insert(0, 'Approver', approver_column)

In [344]:
line_cost_column = df_billable.pop('Line Cost')
df_billable.insert(0, 'Line Cost', line_cost_column)

In [360]:
df_billable
# df_billable["TS_Int"] = pd.to_datetime(df_billable["TS_Int"].dt.strftime("%Y-%m-%d"))
df_billable["TS_Int"] = pd.to_datetime(df_billable["TS_Int"].astype(str), format='%Y-%m-%d')
df_billable["Week Ending"] = pd.to_datetime(df_billable["Week Ending"].astype(str), format='%Y-%m-%d')
df_billable["Week Ending"] = df_billable["Week Ending"].astype(str)
df_billable["TS_Int"] = df_billable["TS_Int"].astype(str)
df_billable['Agreement'] = df_billable['Agreement'].astype(str)


In [361]:
df_billable.to_excel("new_test.xlsx", index=False)